In [2]:
import pandas as pd
import numpy as np
import re
import json
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
# instalar en el ambiente sklearn y pushear a github


### Sistema de Recomendacion


Para la recomendacion, se utilizara un sistema de recomendación basado en el contenido para identificar ítems similares entre si, mediante el seno-coseno mas proximo:

- Representación vectorial: Se representa cada usuario o ítem como un vector de características numéricas.

Estas características pueden ser atributos explícitos (como género de película) o características implícitas derivadas del análisis de datos de interacción (palabras clave extraídas de reseñas).

- Cálculo de similitud coseno: La similitud coseno entre dos vectores se calcula como el coseno del ángulo entre ellos. El coseno del ángulo varía entre -1 y 1, donde:

In [3]:
df = pd.read_parquet('data/df_movies_parquet.parquet', engine='pyarrow')

In [3]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(json.loads)
df['genres'] = df['genres'].apply(json.loads)
df['production_companies'] = df['production_companies'].apply(json.loads)
df['production_countries'] = df['production_countries'].apply(json.loads)
df['spoken_languages'] = df['spoken_languages'].apply(json.loads)

#### Eliminacion de nulos

In [4]:
df = df.dropna(axis=0,subset=['overview'])

Tf-idf (del inglés Term frequency – Inverse document frequency), frecuencia de término – frecuencia inversa de documento (o sea, la frecuencia de ocurrencia del término en la colección de documentos), es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección.

#### Funcion recomendar: recomienda 5 peliculas basadas en el score de similitud
aclaracion: pasar por parametro el nombre exacto de la pelicula

In [5]:
#nltk.download('popular')
stopwords = nltk.corpus.stopwords.words('english') # Guardamos stopwords
lemmatizer = WordNetLemmatizer() # invocamos el lematizador

def preprocesamiento(texto):
    tokens = nltk.word_tokenize(texto.lower())
    # para cada token que cumple la condicion de ser una letra(isalpha()) y no estar dentro de las stopwords
    # se aplica el lematizador y se guarda en tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords]
    return ' '.join(tokens)

df['processed_overview'] = df['overview'].apply(preprocesamiento)
tfidf = TfidfVectorizer(max_df=0.2,max_features=5,sublinear_tf=True)  # Ajustar min_df y max_df según sea necesario
tfidf_matrix = tfidf.fit_transform(df['processed_overview'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['title']).drop_duplicates()
def recomendacion(titulo):
    idx = indices[titulo]
    sim_scores = list(enumerate(cosine_sim[idx])) # retorna una lista con los scores similares
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # ordena las peliculas de puntaje similar
    sim_scores = sim_scores[1:6]  # Obtener los 5 más similares
    movie_indices = [i[0] for i in sim_scores]
    lista_top = df['title'].iloc[movie_indices].tolist()
    return {f'Las recomendaciones para {titulo} son: ':lista_top}
b = recomendacion('Toy Story')
#b = recomendacion('The Magnificent One')
print(b)

{'Las recomendaciones para Toy Story son: ': ['Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the Bride Part II', 'Heat']}


In [6]:
total_letras_process = 0
total_letras = 0
for x in df['overview']:
    total_letras = total_letras + len(x)
print("total de letras sin lematizar: ",total_letras)
for x in df['processed_overview']:
    total_letras_process = total_letras_process + len(x)
print("total de letras lematizadas: ",total_letras_process)

total de letras sin lematizar:  14365729
total de letras lematizadas:  9439527


#### convertir listas a str

In [7]:
def juntar_listas(lista):
  if isinstance(lista, (list)):
    return ','.join([str(elemento[1]) for elemento in lista])
df['generos'] = df['genres'].apply(juntar_listas)
#df["generos"] = df["generos"].apply(lambda x: x.split(", "))

#### funcion para verificar si contiene un genero o no

In [8]:
def contains_genres(genres, genres_to_filter):
    genres_list = [genre.strip() for genre in genres.split(',')]
    return any(genre in genres_list for genre in genres_to_filter)

#### Recomendacion con genero y score de similitud

In [9]:
stopwords = nltk.corpus.stopwords.words('english') # Guardamos stopwords
lemmatizer = WordNetLemmatizer() # invocamos el lematizador

### Funcion para lematizar el texto
def preprocesamiento(texto):
    tokens = nltk.word_tokenize(texto.lower())
    # para cada token que cumple la condicion de ser una letra(isalpha()) y no estar dentro de las stopwords
    # se aplica el lematizador y se guarda en tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords]
    return ' '.join(tokens)

### Funcion para filtrar los las peliculas con generos similares(debe coincidir al menos 1 genero)
def contains_genres(genres, genres_to_filter):
    genres_list = [genre.strip() for genre in genres.split(',')]
    return any(genre in genres_list for genre in genres_to_filter)
df['title'] = df['title'].str.lower()
### Funcion para recomendar peliculas en base a la similitud de las palabras de sus descripciones y genero 
def recomendacion(titulo: str):
    # Procresamiento y validacion de datos
    titulo = str(titulo).strip().lower()
    linea = df[df['title']== titulo]
    generos = linea['generos']
    generos_list = generos.str.split(',')#transformar el str en una lista
    generos_list = list(generos_list.values)
    df_filtrado = df[df['generos'].apply(lambda x: contains_genres(x, generos_list[0]))]
    #filtrar el dataset por las peliculas que compartan 1 genero al menos y dropear duplicados
    df_filtrado.drop_duplicates(subset=['title'],inplace=True)
    # se resetea el indice porque creamos el dataframe filtrado por los generos
    df_filtrado = df_filtrado.reset_index()
    df_filtrado.drop(columns=['index'],inplace=True)
    indices = pd.Series(df_filtrado.index,index=df_filtrado['title']) #Crear los indices del dataframe
    # Procesamiento de texto
    df_filtrado['processed_overview'] = df_filtrado['overview'].apply(preprocesamiento) #se tokeniza el texto
    # creación de matriz TF-IDF
    # max_df = 0.5 equivale a eliminar del modelo los términos que aparecen en más del 50% de los documentos o peliculas.
    # es decir, mientras mas reduzca el parametro "max_df" mayor sera la
    # importancia relativa de las palabras menos frecuentes
    tfidf = TfidfVectorizer(max_df=0.5, max_features=100, sublinear_tf=True)
    # por el hecho del costo computacional que genera esta vectorizacion de palabras se usara max_features=50
    tfidf_matrix = tfidf.fit_transform(df_filtrado['processed_overview'])
    # Calcular la similitud del coseno
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    idx = indices[titulo]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    lista_top = df['title'].iloc[movie_indices].tolist()
    return {f'Las recomendaciones para {titulo} son: ':lista_top}
#b = recomendacion(' diCkson experimental sound film ')
b = recomendacion('the hills have eyes part ii')
#b = recomendacion(titulo='TOY story')
b

C:\Users\Pablo\AppData\Local\Temp\ipykernel_1500\1676335067.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop_duplicates(subset=['title'],inplace=True)


{'Las recomendaciones para the hills have eyes part ii son: ': ['bull durham',
  'the pest',
  'the naked man',
  'bonnie and clyde',
  'anaconda']}